# Обработка датасета и упаковка HDF5

Сбор кучи файлов в одний единый сжатый HDF5

In [1]:
import os
import h5py
import numpy as np
from datetime import datetime, timedelta
from pathlib import Path
from tqdm import tqdm
import json
import time

In [ ]:
source_path = "../mt-grabber/archive_sorted"

In [ ]:
F_VER = "1.0"

positions_dtype = np.dtype([
    ("ship_id", "i8"),           # Внутренний идентификатор судна
    ("timestamp", "i4"),         # Время отчета
    ("lat", "f4"),               # Широта (WGS-84), в градусах
    ("lon", "f4"),               # Долгота (WGS-84), в градусах
    ("speed", "i4"),             # Скорость над грунтом (Speed over ground) в узлах
    ("course", "i4"),            # Курс (Course over ground) (в градусах)
    ("heading", "i4"),           # Направление носа судна (Heading), целое (в градусах)
    ("rot", "i4"),               # Rate of Turn (изменение курса), в AIS шкале (-127..127)
    ("elapsed", "i4"),           # Время с последнего отчета (секунды)
    ("destination", "S64"),      # Указанный порт/пункт назначения (текст, неформализован)
    ("tile_z", "i4"),            # Уровень зума тайла Marine Traffic
    ("file_id", "i4"),           # Индекс файла в таблице `/files`, откуда поступила запись
    ("track_id", "i8"),          # Идентификатор трека (маршрута)
])

ships_dtype = np.dtype([
    ("ship_id", "i8"),           # Внутренний уникальный идентификатор
    ("mt_id", "S128"),           # Marinetraffic ID
    ("name", "S128"),            # Название судна
    ("flag", "S4"),              # ISO-код страны флага (например, "RU", "CN")
    ("ship_type", "i4"),         # AIS raw ship type
    ("gt_ship_type", "i4"),      # Нормализованный/кластеризованный тип судна
    ("length", "i4"),            # Длина судна (в метрах)
    ("width", "i4"),             # Ширина судна (в метрах)
    ("dwt", "i4"),               # Deadweight tonnage - дедвейт, тоннаж
])

tracks_dtype = np.dtype([
    ("track_id", "i8"),          # Уникальный ID трека
    ("ship_id", "i8"),           # Идентификатор судна
    ("start_timestamp", "i4"),   # Время начала трека
    ("end_timestamp", "i4"),     # Время окончания трека
    ("start_lat", "f4"),         # Координаты начальной точки трека
    ("start_lon", "f4"),
    ("end_lat", "f4"),           # Координаты финальной точки трека
    ("end_lon", "f4"),
    ("points_count", "i4"),      # Количество точек в треке
])

files_dtype = np.dtype([
    ("file_id", "i4"),           # Уникальный идентификатор файла
    ("name", "S256"),            # Имя файла
    ("positions_count", "i4"),   # Кол-во записей в файле
    ("timestamp", "i4"),         # Время парсинга
])

In [4]:
# Подсчет размера папки + количества файлов
def get_folder_stats(path):
    total_size = 0
    file_count = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for f in filenames:
            try:
                fp = os.path.join(dirpath, f)
                if os.path.isfile(fp):
                    total_size += os.path.getsize(fp)
                    file_count += 1
            except OSError:
                # Игнорируем ошибки доступа к файлам
                continue
    # Перевод в гигабайты
    size_gb = total_size / (1024 ** 3)
    return size_gb, file_count

source_stats = get_folder_stats(source_path)

def create_hdf5(filename: str = "mt_master.h5"):
    with h5py.File(filename, "w") as h5:
        # Meta
        h5.attrs["created_at"] = datetime.utcnow().isoformat()
        h5.attrs["version"] = F_VER
        h5.attrs["author"] = "Mark Vodyanitskiy (mvodya@icloud.com)"
        h5.attrs["sources_count"] = source_stats[1]
        h5.attrs["sources_size"] = f"{source_stats[0]:.4}Gb"

        # Empty datasets
        h5.create_dataset("ships", shape=(0,), maxshape=(None,), dtype=ships_dtype,
                          chunks=True, compression="gzip", compression_opts=4)

        h5.create_dataset("files", shape=(0,), maxshape=(None,), dtype=files_dtype,
                          chunks=True, compression="gzip", compression_opts=4)

        h5.create_dataset("tracks", shape=(0,), maxshape=(None,), dtype=tracks_dtype,
                          chunks=True, compression="gzip", compression_opts=4)

    print(f"Created HDF5 file: {filename}")

In [5]:
create_hdf5()

Created HDF5 file: mt_master.h5


/var/folders/2c/2p1h64qd4zb1g13m_f5z4z9w0000gn/T/ipykernel_63034/2867499226.py:24: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  h5.attrs["created_at"] = datetime.utcnow().isoformat()


In [6]:
def get_json_files_by_date_range(dir: Path, start_date: str, end_date: str) -> list[Path]:
    root_dir = Path(dir)
    result = []
    date_format = "%d.%m.%Y"
    start = datetime.strptime(start_date, date_format)
    end = datetime.strptime(end_date, date_format)

    current = start
    while current <= end:
        year = f"{current.year:04d}"
        month = f"{current.month:02d}"
        day = f"{current.day:02d}"
        dir_path = root_dir / year / month / day
        if dir_path.exists():
            result.extend(sorted(dir_path.glob("*.json")))
        current += timedelta(days=1)

    return result

In [7]:
start_date = "07.06.2021"
end_date = "10.06.2026"

# Получаем список файлов
files = get_json_files_by_date_range(source_path, start_date, end_date)
files[:2]

[PosixPath('/Volumes/SSD/mark/Documents/Works/MT_Dataset/archive/2024/10/29/29.10.2024_11_26.json'),
 PosixPath('/Volumes/SSD/mark/Documents/Works/MT_Dataset/archive/2024/10/29/29.10.2024_11_35.json')]

In [ ]:
# Маппинг локальных ID с идентификаторами marinetraffic
mt_id_storage = {}
next_ship_id = 1
file_id = 0

# Буферы
ships_batch = []
positions_batch = []
files_batch = []

def safe_int(value, default=-1):
    try:
        return int(value) if value is not None else default
    except (ValueError, TypeError):
        return default

# Сохранение буферов
def flush_to_hdf5(h5file, ships_batch, positions_batch, files_batch):
    if ships_batch:
        ds = h5file["ships"]
        old = ds.shape[0]
        ds.resize((old + len(ships_batch),))
        ds[old:] = np.array(ships_batch, dtype=ships_dtype)
        ships_batch.clear()

    if positions_batch:
        grouped = {}
        for row in positions_batch:
            ts = int(row[1])  # timestamp (2-й столбец)
            dt = datetime.utcfromtimestamp(ts)
            key = (dt.year, dt.month, dt.day)
            grouped.setdefault(key, []).append(row)

        for (y, m, d), rows in grouped.items():
            path = f"positions/{y:04d}/{m:02d}"
            name = f"{d:02d}"
            group = h5file.require_group(path)
            if name not in group:
                ds = group.create_dataset(
                    name,
                    shape=(0,),
                    maxshape=(None,),
                    dtype=positions_dtype,
                    chunks=True,
                    compression="gzip",
                    compression_opts=4,
                )
            else:
                ds = group[name]
            old = ds.shape[0]
            ds.resize((old + len(rows),))
            ds[old:] = np.array(rows, dtype=positions_dtype)

        positions_batch.clear()

    if files_batch:
        ds = h5file["files"]
        old = ds.shape[0]
        ds.resize((old + len(files_batch),))
        ds[old:] = np.array(files_batch, dtype=files_dtype)
        files_batch.clear()
    
    h5file.flush()

flush_every = 50

# Открываем и работаем с HDF файлом
with h5py.File("mt_master.h5", "a") as h5file:
    # Перебираем файлы
    for file in tqdm(files, desc="Parsing JSON's"):
        with open(file, 'r') as f:
            data = json.load(f)
            count = len(data)
            files_batch.append((
                file_id,
                file.name,
                count,
                int(time.time())
            ))
            # Перебираем записи
            for mt_id, record in data.items():
                # Если судно еще не зарегистрированно - регистрируем
                if mt_id not in mt_id_storage:
                    mt_id_storage[mt_id] = next_ship_id
                    next_ship_id += 1
                    # Добавляем судно
                    ships_batch.append((
                        mt_id_storage[mt_id],
                        mt_id,
                        record.get("SHIPNAME", "null"),
                        record.get("FLAG", "null"),
                        safe_int(record.get("SHIPTYPE", -1)),
                        safe_int(record.get("GT_SHIPTYPE", -1)),
                        safe_int(record.get("LENGTH", -1)),
                        safe_int(record.get("WIDTH", -1)),
                        safe_int(record.get("DWT", -1)),
                    ))
                ship_id = mt_id_storage[mt_id]

                # Добываем данные о позиции
                lat = float(record["LAT"])
                lon = float(record["LON"])
                tile_z = int(record["TILE_Z"])

                # Добавляем точку
                positions_batch.append((
                    ship_id,
                    float(record["TIMESTAMP"]),
                    lat,
                    lon,
                    safe_int(record.get("SPEED", -1)),
                    safe_int(record.get("COURSE", -1)),
                    safe_int(record.get("HEADING", -1)),
                    safe_int(record.get("ROT", 0)),
                    safe_int(record.get("ELAPSED", 0)),
                    record.get("DESTINATION", "null"),
                    tile_z,
                    file_id,
                    -1, # track_id = unset
                ))
        file_id += 1

        # Сброс буфферов в HDF
        if file_id % flush_every == 0:
            flush_to_hdf5(h5file, ships_batch, positions_batch, files_batch)
    
    # Финальный сброс буфферов в HDF
    flush_to_hdf5(h5file, ships_batch, positions_batch, files_batch)


Parsing JSON's:   0%|          | 49/27549 [00:06<1:01:19,  7.47it/s]/var/folders/2c/2p1h64qd4zb1g13m_f5z4z9w0000gn/T/ipykernel_63034/1754752334.py:28: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(ts)
Parsing JSON's: 100%|██████████| 27549/27549 [3:07:42<00:00,  2.45it/s]   
